In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3, VGG16, MobileNetV2, Xception, NASNetLarge, DenseNet201, DenseNet121
from sklearn.metrics import mean_squared_error

ImportError: Traceback (most recent call last):
  File "c:\Users\nesri\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
import pandas as pd

file_path = r"parkinsons_updrs.data"
df = pd.read_csv(file_path)
print(df.head())

             name  MDVP:Fo(Hz)  MDVP:Fhi(Hz)  MDVP:Flo(Hz)  MDVP:Jitter(%)  \
0  phon_R01_S01_1      119.992       157.302        74.997         0.00784   
1  phon_R01_S01_2      122.400       148.650       113.819         0.00968   
2  phon_R01_S01_3      116.682       131.111       111.555         0.01050   
3  phon_R01_S01_4      116.676       137.871       111.366         0.00997   
4  phon_R01_S01_5      116.014       141.781       110.655         0.01284   

   MDVP:Jitter(Abs)  MDVP:RAP  MDVP:PPQ  Jitter:DDP  MDVP:Shimmer  ...  \
0           0.00007   0.00370   0.00554     0.01109       0.04374  ...   
1           0.00008   0.00465   0.00696     0.01394       0.06134  ...   
2           0.00009   0.00544   0.00781     0.01633       0.05233  ...   
3           0.00009   0.00502   0.00698     0.01505       0.05492  ...   
4           0.00011   0.00655   0.00908     0.01966       0.06425  ...   

   Shimmer:DDA      NHR     HNR  status      RPDE       DFA   spread1  \
0      0.0654

In [3]:
# Separate features (X) and target (y)
X = df.drop(['name', 'total_UPDRS'], axis=1)  
y = df['total_UPDRS']
X, y

NameError: name 'df' is not defined

In [4]:
scaler = StandardScaler()

# Fit and transform the features (X)
X_scaled = scaler.fit_transform(X)

# Now X_scaled contains the scaled features, and y contains the target values
print("Shape of X_scaled:", X_scaled.shape)
print("Shape of y:", y.shape)

Shape of X_scaled: (195, 22)
Shape of y: (195,)


In [5]:
X_scaled.shape

(195, 22)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Check shape first
print("Original X_reshaped shape:", X_reshaped.shape)

# Add channel if missing
if X_reshaped.ndim == 3:
    X_reshaped = np.expand_dims(X_reshaped, axis=-1)

desired_size = 75
new_shape = (X_reshaped.shape[0], desired_size, desired_size, X_reshaped.shape[3])
X_reshaped_padded = np.zeros(new_shape)

# Calculate centering
start_row = (desired_size - X_reshaped.shape[1]) // 2
start_col = (desired_size - X_reshaped.shape[2]) // 2

# Place original image into center of padded one
X_reshaped_padded[:, start_row:start_row+X_reshaped.shape[1],
                      start_col:start_col+X_reshaped.shape[2], :] = X_reshaped

print("Shape of X_reshaped_padded:", X_reshaped_padded.shape)

# Split
X_train_final, X_test_final, y_train, y_test = train_test_split(
    X_reshaped_padded, y, test_size=0.2, random_state=42)

print("Shape of X_train_final:", X_train_final.shape)
print("Shape of X_test_final:", X_test_final.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_reshaped: (195, 66, 66, 3)


In [7]:
import numpy as np

# Assuming X_reshaped is your reshaped and upsampled data with shape (1030, 330, 330, 3)
desired_size = 75

# Create a new array of zeros with the desired shape
new_shape = (X_reshaped.shape[0], desired_size, desired_size, X_reshaped.shape[3])
X_reshaped_padded = np.zeros(new_shape)

# Calculate the starting indices to copy the original data into the padded array
start_row = (desired_size - X_reshaped.shape[1]) // 2
start_col = (desired_size - X_reshaped.shape[2]) // 2

# Copy the original data into the padded array
X_reshaped_padded[:, start_row:start_row+X_reshaped.shape[1], start_col:start_col+X_reshaped.shape[2], :] = X_reshaped

print("Shape of X_reshaped_padded:", X_reshaped_padded.shape)


# Split the reshaped and scaled data into training and testing sets
X_train_final, X_test_final, y_train, y_test = train_test_split(
    X_reshaped_padded, y, test_size=0.2, random_state=42
)

print("Shape of X_train_final:", X_train_final.shape)
print("Shape of X_test_final:", X_test_final.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_reshaped_padded: (195, 75, 75, 3)
Shape of X_train_final: (156, 75, 75, 3)
Shape of X_test_final: (39, 75, 75, 3)
Shape of y_train: (156,)
Shape of y_test: (39,)


In [8]:
def reshape_split(X, y, dim):
    desired_size = dim - (dim % X.shape[1])
    num_channels = 3

    # Calculate the number of repeats needed to match the desired size
    num_repeats = (desired_size * desired_size * num_channels) // X_scaled.shape[1]

    # Reshape and upsample the scaled input data to match the desired shape
    X_reshaped = np.repeat(X_scaled[:, np.newaxis, :], num_repeats, axis=1)
    X_reshaped = X_reshaped.reshape(X_scaled.shape[0], desired_size, desired_size, num_channels)

    print("Shape of X_reshaped:", X_reshaped.shape)
    
    desired_size = dim
    # Create a new array of zeros with the desired shape
    new_shape = (X_reshaped.shape[0], desired_size, desired_size, X_reshaped.shape[3])
    X_reshaped_padded = np.zeros(new_shape)

    # Calculate the starting indices to copy the original data into the padded array
    start_row = (desired_size - X_reshaped.shape[1]) // 2
    start_col = (desired_size - X_reshaped.shape[2]) // 2

    # Copy the original data into the padded array
    X_reshaped_padded[:, start_row:start_row+X_reshaped.shape[1], start_col:start_col+X_reshaped.shape[2], :] = X_reshaped

    print("Shape of X_reshaped_padded:", X_reshaped_padded.shape)

    # Split the reshaped and scaled data into training and testing sets
    X_train_final, X_test_final, y_train, y_test = train_test_split(
        X_reshaped_padded, y, test_size=0.2, random_state=42
    )

    print("Shape of X_train_final:", X_train_final.shape)
    print("Shape of X_test_final:", X_test_final.shape)
    print("Shape of y_train:", y_train.shape)
    print("Shape of y_test:", y_test.shape)
    
    return X_train_final, X_test_final, y_train, y_test
    

In [9]:
X_train_final, X_test_final, y_train, y_test = reshape_split(X, y, 75)

Shape of X_reshaped: (195, 66, 66, 3)
Shape of X_reshaped_padded: (195, 75, 75, 3)
Shape of X_train_final: (156, 75, 75, 3)
Shape of X_test_final: (39, 75, 75, 3)
Shape of y_train: (156,)
Shape of y_test: (39,)


In [10]:
def f1(p, r):
    return (2 * p * r) / (p+r)

In [11]:
import time
start = time.time()
end = time.time()
print(f"Computational time {end - start} seconds")

Computational time 2.5272369384765625e-05 seconds


In [12]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

# Load the pre-trained VGG16 model (excluding the top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(75, 75, 3))

# Freeze the base model layers
base_model.trainable = False

# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)

print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7110 - auc: 0.5311 - loss: 0.6036 - precision: 0.7110 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.8929 - val_loss: 0.4075 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 837ms/step - accuracy: 0.7466 - auc: 0.8185 - loss: 0.5124 - precision: 0.7466 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.8393 - val_loss: 0.4410 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 782ms/step - accuracy: 0.7353 - auc: 0.8550 - loss: 0.4843 - precision: 0.7353 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.8571 - val_loss: 0.3726 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 782ms/step - accuracy: 0.7383 - auc: 0.9130 - loss: 0.4333 - precision: 0.7383 - recall: 1.0000 - val_accuracy: 0.8125 - val_auc: 0.8571 - val_loss: 0.3759 - val_precision: 0.8667 - val_recall: 0.9286
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 829ms/ste

In [13]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)
loss, accuracy = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)

print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

/tmp/ipykernel_18/2097505671.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(75, 75, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 447ms/step - accuracy: 0.4905 - loss: 3.0850 - val_accuracy: 0.8750 - val_loss: 0.5087
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.7322 - loss: 0.8003 - val_accuracy: 0.8750 - val_loss: 0.4339
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.6888 - loss: 0.6763 - val_accuracy: 0.8750 - val_loss: 0.5890
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.7127 - loss: 0.6418 - val_accuracy: 0.8750 - val_loss: 0.5468
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.7088 - loss: 0.5879 - val_accuracy: 0.8750 - val_loss: 0.4345
Epoch 6/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.7361 - loss: 0.5455 - val_accuracy: 0.8750 - val_loss: 0.4277
Epoch 7/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.7170 - loss: 0.5529 - val_accuracy: 0.8750 - val_loss: 0.4445
Epoch 8/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.7578 - loss: 0.4824 - val_accuracy: 0.8750 - val_loss: 0.4630
2/2 ━━━

In [14]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)

print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ ?                      │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21,802,784 (83.17 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 961ms/step - accuracy: 0.6934 - auc: 0.5145 - loss: 0.7287 - precision: 0.7080 - recall: 0.9622 - val_accuracy: 0.8750 - val_auc: 0.6071 - val_loss: 0.3740 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step - accuracy: 0.7348 - auc: 0.6928 - loss: 0.5069 - precision: 0.7348 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.7857 - val_loss: 0.2992 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - accuracy: 0.7340 - auc: 0.7796 - loss: 0.4875 - precision: 0.7340 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.7857 - val_loss: 0.3877 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.7231 - auc: 0.7730 - loss: 0.4475 - precision: 0.7231 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.7321 - val_loss: 0.3563 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms

In [15]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ ?                      │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 953ms/step - accuracy: 0.3618 - auc: 0.4757 - loss: 0.7028 - precision: 0.3802 - recall: 0.1537 - val_accuracy: 0.8750 - val_auc: 0.5714 - val_loss: 0.4137 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.7131 - auc: 0.4458 - loss: 0.6155 - precision: 0.7131 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.6429 - val_loss: 0.4288 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - accuracy: 0.7466 - auc: 0.4498 - loss: 0.5720 - precision: 0.7466 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.4384 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 207ms/step - accuracy: 0.7040 - auc: 0.5073 - loss: 0.6103 - precision: 0.7040 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.2143 - val_loss: 0.4717 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms

In [16]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = Xception(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ ?                      │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,861,480 (79.58 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 802ms/step - accuracy: 0.6213 - auc: 0.3912 - loss: 1.5440 - precision: 0.6927 - recall: 0.8374 - val_accuracy: 0.1250 - val_auc: 0.1786 - val_loss: 0.8948 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 379ms/step - accuracy: 0.4353 - auc: 0.6180 - loss: 0.7310 - precision: 0.6711 - recall: 0.2996 - val_accuracy: 0.8750 - val_auc: 0.7143 - val_loss: 0.5627 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 380ms/step - accuracy: 0.7088 - auc: 0.7386 - loss: 0.5984 - precision: 0.7088 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.8571 - val_loss: 0.3662 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 373ms/step - accuracy: 0.7196 - auc: 0.6587 - loss: 0.5337 - precision: 0.7196 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.8929 - val_loss: 0.3103 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 

In [17]:
import tensorflow as tf
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

343610240/343610240 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ NASNet (Functional)             │ ?                      │    84,916,818 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,916,818 (323.93 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 84,916,818 (323.93 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 49s 4s/step - accuracy: 0.5706 - auc: 0.5443 - loss: 0.6709 - precision: 0.7836 - recall: 0.6135 - val_accuracy: 0.8750 - val_auc: 0.6250 - val_loss: 0.4865 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.7748 - auc: 0.5579 - loss: 0.5547 - precision: 0.7748 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.6429 - val_loss: 0.4085 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7149 - auc: 0.7592 - loss: 0.5462 - precision: 0.7149 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5536 - val_loss: 0.4872 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 999ms/step - accuracy: 0.8189 - auc: 0.9184 - loss: 0.4881 - precision: 0.8019 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5714 - val_loss: 0.4704 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - a

In [18]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet201 (Functional)        │ ?                      │    18,321,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_6      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,321,984 (69.89 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 18,321,984 (69.89 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.6245 - auc: 0.5583 - loss: 1.2112 - precision: 0.7464 - recall: 0.7441 - val_accuracy: 0.6250 - val_auc: 0.7500 - val_loss: 0.6568 - val_precision: 1.0000 - val_recall: 0.5714
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 406ms/step - accuracy: 0.6910 - auc: 0.6821 - loss: 0.5821 - precision: 0.7588 - recall: 0.8539 - val_accuracy: 0.8750 - val_auc: 0.8571 - val_loss: 0.3401 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 401ms/step - accuracy: 0.7151 - auc: 0.7881 - loss: 0.5343 - precision: 0.7429 - recall: 0.9303 - val_accuracy: 0.7500 - val_auc: 0.7857 - val_loss: 0.5804 - val_precision: 1.0000 - val_recall: 0.7143
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 424ms/step - accuracy: 0.7682 - auc: 0.8389 - loss: 0.4629 - precision: 0.8297 - recall: 0.8839 - val_accuracy: 0.8750 - val_auc: 0.8214 - val_loss: 0.3423 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 425ms/st

In [19]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ ?                      │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,037,504 (26.85 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.4446 - auc: 0.4509 - loss: 0.9379 - precision: 0.6608 - recall: 0.3563 - val_accuracy: 0.8750 - val_auc: 0.2857 - val_loss: 0.6001 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 276ms/step - accuracy: 0.6828 - auc: 0.5649 - loss: 0.6360 - precision: 0.6828 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.6071 - val_loss: 0.5075 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 341ms/step - accuracy: 0.6871 - auc: 0.5812 - loss: 0.9944 - precision: 0.6871 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6596 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 302ms/step - accuracy: 0.6997 - auc: 0.5382 - loss: 0.6595 - precision: 0.6997 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6512 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 319ms/st

In [20]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.5066 - auc: 0.5226 - loss: 0.7143 - precision: 0.5753 - recall: 0.5613 - val_accuracy: 0.8750 - val_auc: 0.6250 - val_loss: 0.3531 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 343ms/step - accuracy: 0.7214 - auc: 0.7332 - loss: 0.5981 - precision: 0.7214 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5179 - val_loss: 0.3810 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 349ms/step - accuracy: 0.7448 - auc: 0.8508 - loss: 0.4911 - precision: 0.7448 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.4216 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 368ms/step - accuracy: 0.7552 - auc: 0.7556 - loss: 0.4996 - precision: 0.7552 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5357 - val_loss: 0.4113 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 368ms/st

In [21]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ ?                      │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_9      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,783,535 (41.14 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 10,783,535 (41.14 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.6715 - auc: 0.5204 - loss: 1.1025 - precision: 0.7285 - recall: 0.8722 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6864 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 345ms/step - accuracy: 0.7227 - auc: 0.4936 - loss: 0.6875 - precision: 0.7227 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6695 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.7175 - auc: 0.4917 - loss: 0.6769 - precision: 0.7175 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6454 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 334ms/step - accuracy: 0.6979 - auc: 0.5252 - loss: 0.6656 - precision: 0.6979 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6187 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 338ms/st

In [22]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet101 (Functional)          │ ?                      │    42,658,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_10     │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,658,176 (162.73 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 42,658,176 (162.73 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.6047 - auc: 0.4993 - loss: 0.6457 - precision: 0.7469 - recall: 0.7217 - val_accuracy: 0.8750 - val_auc: 0.5714 - val_loss: 0.4232 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 611ms/step - accuracy: 0.7118 - auc: 0.7543 - loss: 0.5363 - precision: 0.7118 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.6607 - val_loss: 0.3883 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 672ms/step - accuracy: 0.7144 - auc: 0.8070 - loss: 0.5015 - precision: 0.7144 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.7500 - val_loss: 0.3641 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 689ms/step - accuracy: 0.7227 - auc: 0.9169 - loss: 0.4375 - precision: 0.7227 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.7500 - val_loss: 0.3588 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 680ms/st

In [23]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet152 (Functional)          │ ?                      │    58,370,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_11     │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,370,944 (222.67 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 58,370,944 (222.67 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.6915 - auc: 0.5965 - loss: 0.7583 - precision: 0.7516 - recall: 0.8559 - val_accuracy: 0.8750 - val_auc: 0.6429 - val_loss: 0.3505 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 917ms/step - accuracy: 0.7439 - auc: 0.7096 - loss: 0.5809 - precision: 0.7418 - recall: 0.9851 - val_accuracy: 0.7500 - val_auc: 0.7143 - val_loss: 0.4375 - val_precision: 0.9167 - val_recall: 0.7857
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 951ms/step - accuracy: 0.8389 - auc: 0.9150 - loss: 0.4018 - precision: 0.8282 - recall: 0.9644 - val_accuracy: 0.8125 - val_auc: 0.7143 - val_loss: 0.3428 - val_precision: 0.8667 - val_recall: 0.9286
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 893ms/step - accuracy: 0.8302 - auc: 0.9131 - loss: 0.3484 - precision: 0.8206 - recall: 0.9883 - val_accuracy: 0.6250 - val_auc: 0.7857 - val_loss: 0.6747 - val_precision: 1.0000 - val_recall: 0.5714
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 901ms/st

In [24]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb1 (Functional)     │ ?                      │     6,575,239 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_12     │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,575,239 (25.08 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 6,575,239 (25.08 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.6557 - auc: 0.4721 - loss: 0.8653 - precision: 0.7313 - recall: 0.8366 - val_accuracy: 0.8750 - val_auc: 0.4286 - val_loss: 0.4626 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - accuracy: 0.7331 - auc: 0.5017 - loss: 0.5934 - precision: 0.7331 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5714 - val_loss: 0.4225 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 232ms/step - accuracy: 0.7166 - auc: 0.4481 - loss: 0.6041 - precision: 0.7166 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.4643 - val_loss: 0.4634 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - accuracy: 0.7118 - auc: 0.5177 - loss: 0.6008 - precision: 0.7118 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.1786 - val_loss: 0.4520 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 229ms/st

In [25]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
start = time.time()

base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
base_model.trainable = False
# Build the binary classification model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(16, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Define the desired learning rate (e.g., 0.01)
learning_rate = 0.01

# Compile the model with the specified learning rate and binary_crossentropy loss
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Display a summary of the model architecture
model.summary()

# Train the model
history = model.fit(X_train_final, y_train, epochs=8, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy, precision, recall, auc = model.evaluate(X_test_final, y_test)
print("Loss on Test Set:", loss)
print("Accuracy on Test Set:", accuracy)
print("precision on Test Set:", precision)
print("recall on Test Set:", recall)
print("f1 score on Test Set:", f1(precision, recall))
print("auc on Test Set:", auc)


print("Error rate on Test Set:", 1 -accuracy)
end = time.time()
print(f"Computational time {end - start} seconds")

/opt/conda/lib/python3.10/site-packages/keras/src/applications/mobilenet_v3.py:449: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ MobilenetV3small (Functional)   │ ?                      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_13     │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 939,120 (3.58 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 939,120 (3.58 MB)

Epoch 1/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 474ms/step - accuracy: 0.6941 - auc: 0.4859 - loss: 0.8243 - precision: 0.7552 - recall: 0.8653 - val_accuracy: 0.1250 - val_auc: 0.5000 - val_loss: 0.6955 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4238 - auc: 0.5080 - loss: 0.6937 - precision: 0.4748 - recall: 0.3221 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6865 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7040 - auc: 0.5179 - loss: 0.6876 - precision: 0.7040 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6721 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 4/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.7335 - auc: 0.4832 - loss: 0.6784 - precision: 0.7335 - recall: 1.0000 - val_accuracy: 0.8750 - val_auc: 0.5000 - val_loss: 0.6569 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 5/8
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5